<a href="https://colab.research.google.com/github/JyotiRawat29/MastersThesis/blob/main/Base_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from torch import Tensor
import torch.nn.functional as f
from torch import nn
import scipy.io as sio
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader
from multiprocessing import cpu_count
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from tensorflow import summary
import matplotlib.pyplot as plt

In [ ]:
writer = SummaryWriter()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.cuda.empty_cache()
#load the ECG file
src_dataset = sio.loadmat('/content/drive/MyDrive/ECG2(withDA).mat')
#load the labels
label=pd.read_csv('/content/drive/MyDrive/label.csv',header=None)

In [ ]:
testdata = src_dataset['ECG'] # use the key for data here
X=testdata['Data'] #loading data

In [ ]:
X=np.array(X[0])
X = np.vstack(X[:,]).astype(np.float)
X=torch.from_numpy(X)

In [ ]:
label=np.array(label, dtype=object) # loading labels 
label=np.array(label)
label= np.vstack(label[:,]).astype(np.float)
Y = torch.tensor(label)

In [ ]:
print(X.shape)
print(Y.shape)

torch.Size([1273, 4000])
torch.Size([1273, 1])


In [ ]:
def create_datasets(X, y, test_size=0.2, time_dim_first=False):
    enc = LabelEncoder()
    y_enc = enc.fit_transform(y)
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, stratify=Y)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=1)  
    print(X_train.shape)
    print(X_valid.shape)
    print(y_train.shape)
    y_train=torch.squeeze(y_train)
    print(y_test.shape)
    y_valid=torch.squeeze(y_valid)
    y_test=torch.squeeze(y_test)

    X_train, X_valid, X_test = [torch.tensor(arr, dtype=torch.float32) for arr in (X_train, X_valid,X_test)]
    y_train, y_valid,y_test = [torch.tensor(arr, dtype=torch.long) for arr in (y_train, y_valid,X_test)]
    print(X_train.shape)
    print(X_valid.shape)
    print(X_test.shape)
    train_ds = TensorDataset(X_train, y_train)
    valid_ds = TensorDataset(X_valid, y_valid)
    tst_ds=TensorDataset(X_test,y_test)
    return train_ds, valid_ds,tst_ds, enc

In [ ]:
trn_ds, val_ds, tst_data, enc = create_datasets(X, Y)

torch.Size([763, 4000])
torch.Size([255, 4000])
torch.Size([763, 1])
torch.Size([255, 1])
torch.Size([763, 4000])
torch.Size([255, 4000])
torch.Size([255, 4000])


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


In [ ]:
def create_loaders(train_ds, valid_ds,tst_ds, bs=512, jobs=0):
    train_dl = DataLoader(train_ds, bs, shuffle=True, num_workers=jobs)
    valid_dl = DataLoader(valid_ds, bs, shuffle=False, num_workers=jobs)
    tst_dl = DataLoader(tst_ds, bs, shuffle=False, num_workers=jobs)
    return train_dl, valid_dl,tst_ds

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size,batch_first=True)

        self.linear = nn.Linear(hidden_layer_size, output_size)
        
        self.fc = nn.Linear(hidden_layer_size, output_size)


        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        h0 = torch.zeros(1, input_seq.size(0), self.hidden_layer_size).to(device) 
        c0 = torch.zeros(1, input_seq.size(0), self.hidden_layer_size).to(device)
        lstm_out, _ = self.lstm(input_seq, (h0,c0))
        lstm_out = self.fc(lstm_out[:, -1, :])
        #predictions = self.linear(lstm_out.view(len(input_seq), -1))
        #print("predictions",predictions)
        return lstm_out

    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        
        #print('hidden layer')
        #print(h0.shape)
        #print(x.size(0))
        #print(layer_dim)
        return [t.to(device) for t in (h0, c0)]

In [ ]:
trn_ds, val_ds, tst_data, enc = create_datasets(X, Y)
bs =50
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


torch.Size([763, 4000])
torch.Size([255, 4000])
torch.Size([763, 1])
torch.Size([255, 1])
torch.Size([763, 4000])
torch.Size([255, 4000])
torch.Size([255, 4000])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


In [ ]:
print(f'Creating data loaders with batch size: {bs}')
trn_dl, val_dl,tst_data = create_loaders(trn_ds, val_ds,tst_data, bs, jobs=cpu_count())
model = LSTM(1,500,1)
layer_dim = 1

Creating data loaders with batch size: 50


In [ ]:
!pip install torchinfo

In [ ]:
from tensorflow import summary
from torchinfo import summary

In [ ]:
summary(model)

Layer (type:depth-idx)                   Param #
LSTM                                     --
├─LSTM: 1-1                              1,006,000
├─Linear: 1-2                            501
├─Linear: 1-3                            501
Total params: 1,007,002
Trainable params: 1,007,002
Non-trainable params: 0

In [ ]:
model = model.to(device)

In [ ]:
seq_dim = 4000
lr = 0.001
n_epochs = 101
iterations_per_epoch = len(trn_dl)
best_acc = 0
patience, trials = 100, 0
loss_function = nn.CrossEntropyLoss()
layer_dim = 1
output_dim = 3

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import copy

In [ ]:
best_loss = 1
n_epochs = 100
best_model_wts = copy.deepcopy(model.state_dict())

In [ ]:
history = dict(train =[], val = [])

In [ ]:
for epoch in range(1, n_epochs + 1):
  train_losses = []
  val_losses = []
  #training loop
  for i, (x_batch, y_batch) in enumerate(trn_dl):
        model.to(device)
        model.train()
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        opt.zero_grad()
        x_batch=torch.unsqueeze(x_batch,2)

        
        out = model(x_batch)
        y_batch=torch.unsqueeze(y_batch,0)
        y_batch = y_batch.to(torch.float32)

        out = out.to(torch.float32)
        out=torch.transpose(out,1,0)
        loss = loss_function(out, torch.max(y_batch, 1)[1])

        loss.backward()
        opt.step()
        train_losses.append(loss.item())
 

  correct, total = 0, 0
  #validation loop
  for x_val, y_val in val_dl:
        model.eval()
        x_val, y_val = [t for t in (x_val, y_val)]
        x_val = x_val.to(device)
        y_val = y_val.to(device)
        x_val=torch.unsqueeze(x_val,2)

        out = model(x_val)
        y_val=torch.unsqueeze(y_val,0)
        y_val = y_val.to(torch.float32)

        out = out.to(torch.float32)
        out=torch.transpose(out,1,0)
        loss = loss_function(out, torch.max(y_val, 1)[1])
        val_losses.append(loss.item())
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        print("y_valsize",y_val.size(0))
        print("y_valsize",y_val.size(1))
        total += y_val.size(0)
        print('total',total)
        correct += (preds == y_val).sum().item()
        print('correct',correct)

  
  train_loss = np.mean(train_losses)

  val_loss = np.mean(val_losses)

  history['train'].append(train_loss)
  history['val'].append(val_loss)
  print("correct",correct)
  print("total",total)

  acc = correct / total

  #if epoch % 5 == 0:
  print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

  #if acc > best_acc:
   #     trials = 0
    #    best_acc = acc
     #   torch.save(model.state_dict(), 'best.pth')
      #  print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
  #else:
   #     trials += 1
    #    if trials >= patience:
     #       print(f'Early stopping on epoch {epoch}')
      #      break
#writer.flush()
#writer.add_graph(model,x_batch)
#writer.close()

y_valsize 1
y_valsize 50
total 1
correct 0
y_valsize 1
y_valsize 50
total 2
correct 0
y_valsize 1
y_valsize 50
total 3
correct 0
y_valsize 1
y_valsize 50
total 4
correct 0
y_valsize 1
y_valsize 50
total 5
correct 13
y_valsize 1
y_valsize 5
total 6
correct 14
correct 14
total 6
Epoch:   1. Loss: 1.6160. Acc.: 233.33%


KeyboardInterrupt: ignored

In [ ]:
test_dl = DataLoader(tst_data, batch_size=64, shuffle=False)
test = []
for batch, _ in tst_data:
    batch=batch.to(device)
    batch = batch.reshape(1,4000,1)
    out = model.train()(batch)
    y_hat = F.log_softmax(out, dim=1).argmax(dim=1)
    test += y_hat.tolist()

In [ ]:
ax = plt.figure().gca()

ax.plot(history['train'])
ax.plot(history['val'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'])
plt.title('Loss over epochs')
#plt.show()
plt.savefig('Loss_over_epochs.png')